In [1]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz
import pandas as pd
import numpy as np


from vispy import app, gloo, visuals, scene
from vispy.color import Color

from vispy.visuals.transforms import MatrixTransform

import sys
# change this to suit your path
sys.path.append('/home/guillaume/Python/kt_simul/')
#from kt_simul.smp_spindle.smp_spindle import SympySpindle, parameters


import IPython.display as disp
import os
import matplotlib.pylab as plt
%matplotlib qt


### Parameters 

In [22]:
from kt_simul.core.components import Structure, Point, Link
from kt_simul.core.dynamics import Model, dampedspring, viscous
from kt_simul.core.dynamics import spring, dashpot, linear_fv, contraction

import pandas as pd

from numpy.testing import assert_almost_equal
# This is an example test, not functional yet, should
# put some effort in implementing for all forces


def test_spring():

    sprg = Structure('simple spring')
    p0 = Point(0, sprg)
    sprg.add_point(p0, pos0=[0, 1, 0])
    p1 = Point(1, sprg)
    sprg.add_point(p1, pos0=[2, 0, 0])
    lnk = sprg.add_link(p0, p1)
    sprg.update_geometry()
    m = Model(sprg)
    viscous(m, p0, 1)
    viscous(m, p1, 1)
    sprg.point_hist = pd.Panel({0: sprg.point_df})

    def model_update(step):

        m.Bvect *= 0
        spring(m, lnk, 0.1, 1)
        sprg.register_history(step)

    for i in range(30):
        m.solve()
        model_update(i)

    assert_almost_equal(p0.dist(p1), 1, decimal=2)
    

def test_dashpot():

    sprg = Structure('simple spring')
    p0 = Point(0, sprg)
    sprg.add_point(p0, pos0=[0, 1, 0])
    p1 = Point(1, sprg)
    sprg.add_point(p1, pos0=[2, 0, 0])
    lnk = sprg.add_link(p0, p1)
    sprg.update_geometry()
    m = Model(sprg)
    viscous(m, p0, 0.01)
    viscous(m, p1, 0.01)
    sprg.point_hist = pd.Panel({0: sprg.point_df})
    
    def model_update(step):

        m.Bvect *= 0

        dampedspring(m, lnk, 2, 1, 0.5)
        sprg.register_history(step)

    for i in range(300):
        m.solve()
        model_update(i)
    return sprg
    
fig, ax = plt.subplots()
colors = ['r', 'b']
for n, p in sprg.points.items():
    ax.plot(p.traj.x, p.traj.y, colors[n]+'o-')
    

In [23]:

sprg = test_dashpot()

In [24]:
fig, ax = plt.subplots()
colors = ['r', 'b']
for n, p in sprg.points.items():
    ax.plot(p.traj.x, colors[n]+'o-')
    ax.plot(p.traj.y, colors[n]+'v-')
    ax.plot(p.traj.z, colors[n]+'^-')

In [26]:
fig, ax = plt.subplots()
colors = ['r', 'b']
for n, p in sprg.points.items():
    ax.plot(p.traj.x, p.traj.y, colors[n]+'o-')


In [25]:
sprg.points[0].dist(sprg.points[1])

0.56635318743695606

In [14]:

p.traj.x

0     2
1     2
2     2
3     2
4     2
5     2
6     2
7     2
8     2
9     2
10    2
11    2
12    2
13    2
14    2
15    2
16    2
17    2
18    2
19    2
20    2
21    2
22    2
23    2
24    2
25    2
26    2
27    2
28    2
29    2
Name: x, dtype: float64

In [3]:
from kt_simul.core.simul_spindle import Metaphase

In [4]:
meta = Metaphase(initial_plug='amphitelic')



In [5]:

meta.simul()

In [7]:
fig = meta.show()

In [17]:
import vispy.io as vp_io

In [22]:
'spindle_{0:03d}.png'.format(10)

'spindle_010.png'

In [26]:
img = win.render()

In [29]:
img.shape

(800, 800, 4)

In [11]:
"""
Simple demonstration of Box visual.
"""
from vispy import scene

spindle = meta.spindle
spindle.point_df['radius'] = 0.1
spindle.point_df['color'] = '#a1a1a1'


spindle.point_df.loc[spindle.spbR.idx, 'radius'] = 0.1
spindle.point_df.loc[spindle.spbR.idx, 'color'] = '#000000'
spindle.point_df.loc[spindle.spbL.idx, 'radius'] = 0.1
spindle.point_df.loc[spindle.spbL.idx, 'color'] = '#000000'



ch_colors = ['#00ffaa', '#5500cc', '#77aaee']
for ch, color in zip(spindle.chromosomes, ch_colors):
    spindle.point_df.loc[ch.cen_A.idx, 'radius'] = 0.2
    spindle.point_df.loc[ch.cen_A.idx, 'color'] = color
    spindle.point_df.loc[ch.cen_B.idx, 'radius'] = 0.2
    spindle.point_df.loc[ch.cen_B.idx, 'color'] = color


class VispyOrganite():
    
    def __init__(self, point, spindle):
        self.point = point
        radius, color = spindle.point_df.loc[point.idx, ['radius', 'color']]
        self.sphere = scene.visuals.Sphere(radius=radius, method='ico', 
                                           subdivisions=4, color=color, parent=view.scene)

        self.transform = MatrixTransform()
        self.sphere.transform = self.transform
        self.traj = point.traj * 0.1
        
        
class VispyLink():
    
    def __init__(self, link, spindle):
        visuals.LineVisual()

from vispy.color import Color
canvas = scene.SceneCanvas(keys='interactive', size=(800, 600), show=True)

# Set up a viewbox to display the cube with interactive arcball
view = canvas.central_widget.add_view()
view.bgcolor = '#efefef'
view.camera = 'turntable'
view.padding = 100

color = Color("#3f51b5")
objects = []
for point in spindle.points.values():
    objects.append(VispyOrganite(point, spindle))
t = 0
    
def move(app):
    global t
    if t >= spindle.point_hist.shape[0]:
        t = 0
    for obj in objects:
        obj.sphere.transform.reset()
        x, y, z = obj.traj.loc[t]
        obj.sphere.transform.translate([x, y, z])#, z))
    img = canvas.render()
    vp_io.write_png('movie/spindle_{0:03d}.png'.format(t), img)
    t += 1
        
timer = app.Timer(connect=move)
timer.start(0.1)

In [14]:
app.set_interactive(True)


In [8]:
img = canvas.render()

In [10]:
vp_io.write_png('test.png', img)

In [79]:
visuals.SphereVisual?

In [43]:
from kt_simul.io.parameters_io import ParamTree
from kt_simul.core.simul_spindle import Metaphase

In [24]:
img

NameError: name 'img' is not defined

ERROR: Invoking <bound method Canvas.move of <Canvas (PyQt4) at 0x7f430cc8ce48>> repeat 256
ERROR:vispy:Invoking <bound method Canvas.move of <Canvas (PyQt4) at 0x7f430cc8ce48>> repeat 256
